# Objectif du Notebook
L'objectif de ce notebook est de manipuler et préparer les données des ventes enregistrées dans une base de données, afin de les rendre prêtes pour des analyses ou des modèles de machine learning. Les données proviennent de différentes tables contenant des informations sur les ventes, les produits, les employés et les régions.

# Connexion à une base de données et Manipulation de données
importation de modules et configuration de la connexion
et filtrage des colonnes et affichage des résultats

In [6]:
import pyodbc
import pandas as pd

# Configuration de la chaîne de connexion
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"  # Remplacez par votre pilote installé
    "Server=DESKTOP-1RQBJ5I;"  # Remplacez par le nom de votre serveur
    "Database=AVW_DW;"  # Nom de votre base de données
    "Trusted_Connection=yes;"  # Utilisation de l'authentification Windows
)

# Fonction pour récupérer les données de la base de données SQL dans un dictionnaire de DataFrames
def fetch_all_tables_from_sql():
    try:
        # Établir une connexion à la base de données
        with pyodbc.connect(conn_str) as conn:
            print("Connexion réussie à la base de données AVW_DW !")
            
            # Dictionnaire des requêtes SQL pour chaque table
            query_dict = {
                "Dim_Year": "SELECT * FROM Dim_Year",
                "Dim_Month": "SELECT * FROM Dim_Month",
                "Dim_Date": "SELECT * FROM Dim_Date",
                "Dim_SalesPerson": "SELECT * FROM Dim_SalesPerson",
                "Dim_Territory": "SELECT * FROM Dim_Territory",
                "Dim_Product": "SELECT * FROM Dim_Product",
                "Dim_ProductSubCategory": "SELECT * FROM Dim_ProductSubCategory",
                "Dim_ProductCategory": "SELECT * FROM Dim_ProductCategory",
                "Fact_SalesOrder": "SELECT * FROM Fact_SalesOrder",
                "Fact_Product": "SELECT * FROM Fact_Product",
            }
            
            tables = {}
            # Récupérer les tables et les ajouter au dictionnaire
            for table_name, query in query_dict.items():
                tables[table_name] = pd.read_sql(query, conn)
            
            # Retourner le dictionnaire des tables (DataFrames)
            return tables
    
    except pyodbc.Error as e:
        print(f"Erreur de connexion à la base de données : {e}")
    except Exception as e:
        print(f"Erreur lors de la récupération des données : {e}")
    return {}

# Récupérer toutes les tables dans un dictionnaire
tables = fetch_all_tables_from_sql()
# Supprimer la colonne 'IsDeleted' de Dim_Territory avant la fusion
tables["Dim_Territory"] = tables["Dim_Territory"].drop(columns=["IsDeleted"], errors='ignore')

# Fusionner avec Dim_Territory
fact_sales_order = pd.merge(fact_sales_order, tables["Dim_Territory"], how='left', on='TerritoryKey', suffixes=('_fact', '_territory'))

# Vérification si les tables sont correctement chargées
if tables:
    try:
        # Fact_SalesOrder
        fact_sales_order = tables["Fact_SalesOrder"]

        # Joindre toutes les autres tables avec des suffixes pour éviter les conflits de noms de colonnes

        fact_sales_order = pd.merge(fact_sales_order, tables["Dim_Date"], how='left', on='DateKey', suffixes=('_fact', '_date'))
        fact_sales_order = pd.merge(fact_sales_order, tables["Dim_SalesPerson"], how='left', on='SalePersonKey', suffixes=('_fact', '_salesperson'))
        fact_sales_order = pd.merge(fact_sales_order, tables["Dim_Territory"], how='left', on='TerritoryKey', suffixes=('_fact', '_territory'))
        fact_sales_order = pd.merge(fact_sales_order, tables["Dim_Product"], how='left', on='ProductKey', suffixes=('_fact', '_product'))
        fact_sales_order = pd.merge(fact_sales_order, tables["Dim_ProductSubCategory"], how='left', on='ProductSubCategoryKey', suffixes=('_fact', '_productsubcategory'))
        fact_sales_order = pd.merge(fact_sales_order, tables["Dim_ProductCategory"], how='left', on='ProductCategoryKey', suffixes=('_fact', '_productcategory'))

        # Maintenant fact_sales_order contient toutes les informations enrichies
        print("Large table of Fact_SalesOrder:")
        print(fact_sales_order.head())  # Afficher les premières lignes de la table

    except KeyError as e:
        print(f"Clé manquante dans les tables: {e}")
else:
    print("Les tables n'ont pas pu être récupérées.")


Connexion réussie à la base de données AVW_DW !
Clé manquante dans les tables: 'ProductKey'


C:\Users\PE\AppData\Local\Temp\ipykernel_22856\229164351.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables[table_name] = pd.read_sql(query, conn)


In [4]:
fact_sales_order.head()

,Id,DateKey,TerritoryKey,SalePersonKey,Revenue,NumberOrder,IsDeleted_fact,MonthKey,DATE,IsDeleted_date,SalePersonId,FullName,NationalIDNumber,Gender,HireDate,IsDeleted,TerritoryId,Name,ContryRegionCode
0,7393,01102011,5,1,2417.47,1,False,201110,2011-10-01,False,274,Stephen Y Jiang,502097814,M,2011-01-04,False,5,Southeast,US
1,7394,30102012,5,1,90167.33,1,False,201210,2012-10-30,False,274,Stephen Y Jiang,502097814,M,2011-01-04,False,5,Southeast,US
2,7395,30082012,4,1,33206.02,1,False,201208,2012-08-30,False,274,Stephen Y Jiang,502097814,M,2011-01-04,False,4,Southwest,US
3,7396,30082012,2,1,29638.27,1,False,201208,2012-08-30,False,274,Stephen Y Jiang,502097814,M,2011-01-04,False,2,Northeast,US
4,7397,30062013,6,1,17990.96,1,False,201306,2013-06-30,False,274,Stephen Y Jiang,502097814,M,2011-01-04,False,6,Canada,CA


# Exportation des données

Cette section exporte le DataFrame final fusionné dans un fichier Excel.

In [5]:


# Colonnes à conserver
columns_to_keep = [
    "SalePersonKey", "Revenue", "NumberOrder", "DATE", 
    "FullName", "Gender", "HireDate", "Name", "ContryRegionCode"
]

# Garder uniquement les colonnes spécifiées
filtered_data = fact_sales_order[columns_to_keep]

# Enregistrer le nouveau fichier Excel
output_file_path = r"C:\Users\PE\Desktop\machine learning\Sales.xlsx"  # Changez le chemin si nécessaire
filtered_data.to_excel(output_file_path, index=False)

print(f"Fichier filtré enregistré dans : {output_file_path}")


Fichier filtré enregistré dans : C:\Users\PE\Desktop\machine learning\Sales.xlsx
